# YOLO for Satellite and for Drone separately

In [3]:
import pandas as pd
from PIL import Image

# Load the .pkl file
df = pd.read_pickle(r"C:\Users\gnvca\OneDrive\Desktop\JP\Model_Train.pkl")
#df = pd.read_pickle(r"C:\Users\gnvca\OneDrive\Desktop\JP\Model_Val.pkl")

Preprocessing for YOLO

In [5]:

def scale_polygon(polygon, scale=1.15):
    """Scales a polygon outward from its centroid."""
    xs, ys = zip(*polygon)
    cx, cy = sum(xs) / len(xs), sum(ys) / len(ys)
    return [(cx + scale * (x - cx), cy + scale * (y - cy)) for x, y in polygon]

def polygon_to_yolo_bbox(polygon, img_width, img_height):
    """Converts polygon to YOLO normalized bbox."""
    xs, ys = zip(*polygon)
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    
    x_center = ((x_min + x_max) / 2) / img_width
    y_center = ((y_min + y_max) / 2) / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    
    return x_center, y_center, width, height

In [ ]:
import os
import pandas as pd

# Define the root directories for labels
drone_label_dir_train = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03split\drone\dataset\labels\train"
drone_label_dir_val = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03split\drone\dataset\labels\val"
satellite_label_dir_train = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03split\satellite\dataset\labels\train"
satellite_label_dir_val = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03split\satellite\dataset\labels\val"

# Create directories if they do not exist
os.makedirs(drone_label_dir_train, exist_ok=True)
os.makedirs(drone_label_dir_val, exist_ok=True)
os.makedirs(satellite_label_dir_train, exist_ok=True)
os.makedirs(satellite_label_dir_val, exist_ok=True)

image_width, image_height = 512, 512  # Replace with actual size if dynamic

# Load the pickled data files (train and validation)
train_df = pd.read_pickle('Model_Train.pkl')
val_df = pd.read_pickle('Model_Val.pkl')

# Function to process the labels and save to .txt files
def process_labels(df_filtered, label_dir):
    for idx, row in df_filtered.iterrows():
        label_lines = []

        # Process boilers (class 0)
        for poly in row['polygons_boil']:
            scaled_poly = scale_polygon(poly, scale=1.15)
            x, y, w, h = polygon_to_yolo_bbox(scaled_poly, image_width, image_height)
            label_lines.append(f"0 {x:.6f} {y:.6f} {w:.6f} {h:.6f}")

        # Process panels (class 1)
        for poly in row['polygons_pan']:
            scaled_poly = scale_polygon(poly, scale=1.15)
            x, y, w, h = polygon_to_yolo_bbox(scaled_poly, image_width, image_height)
            label_lines.append(f"1 {x:.6f} {y:.6f} {w:.6f} {h:.6f}")

        # Save to .txt file
        label_path = os.path.join(label_dir, f"{row['img_id']}.txt")
        with open(label_path, "w") as f:
            f.write("\n".join(label_lines))

# Process training and validation datasets for drone and satellite
for img_origin in ['D', 'S']:
    # Filter the dataframe based on the image origin
    train_filtered = train_df[train_df['img_origin'] == img_origin]
    val_filtered = val_df[val_df['img_origin'] == img_origin]

    # Set the label directories based on the image origin and train/val
    if img_origin == 'D':
        train_label_dir = drone_label_dir_train
        val_label_dir = drone_label_dir_val
    else:
        train_label_dir = satellite_label_dir_train
        val_label_dir = satellite_label_dir_val

    # Process and save training labels
    process_labels(train_filtered, train_label_dir)

    # Process and save validation labels
    process_labels(val_filtered, val_label_dir)


In [ ]:
import os
import shutil
import pandas as pd

# Load the training and validation DataFrames
df_train = pd.read_pickle(r"C:\Users\gnvca\OneDrive\Desktop\JP\Model_Train.pkl")
df_val = pd.read_pickle(r"C:\Users\gnvca\OneDrive\Desktop\JP\Model_Val.pkl")

# Paths
image_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\images"  # Folder with original images

# Create directories for drone and satellite images
drone_train_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03split\drone\dataset\images\train"
drone_val_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03split\drone\dataset\images\val"
satellite_train_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03split\satellite\dataset\images\train"
satellite_val_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03split\satellite\dataset\images\val"

# Create the directories if they do not exist
os.makedirs(drone_train_dir, exist_ok=True)
os.makedirs(drone_val_dir, exist_ok=True)
os.makedirs(satellite_train_dir, exist_ok=True)
os.makedirs(satellite_val_dir, exist_ok=True)

# Function to transfer images based on filtered dataframe
def transfer_images(df_filtered, output_dir):
    for img_id in df_filtered['img_id']:
        src_path = os.path.join(image_dir, f"{img_id}.jpg")
        dst_path = os.path.join(output_dir, f"{img_id}.jpg")

        if os.path.exists(src_path):
            shutil.copy2(src_path, dst_path)
        else:
            print(f"⚠️ Warning: Image not found: {src_path}")

# Process images for both training and validation datasets
for img_origin in ['D', 'S']:
    # Filter train and val data based on the image origin
    train_filtered = df_train[df_train['img_origin'] == img_origin]
    val_filtered = df_val[df_val['img_origin'] == img_origin]

    # Set the destination directories based on the image origin and train/val
    if img_origin == 'D':
        train_output_dir = drone_train_dir
        val_output_dir = drone_val_dir
    else:
        train_output_dir = satellite_train_dir
        val_output_dir = satellite_val_dir

    # Transfer training images
    transfer_images(train_filtered, train_output_dir)

    # Transfer validation images
    transfer_images(val_filtered, val_output_dir)


Training YOLO

In [ ]:
import os

label_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\labels\train"
image_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\train"

label_files = set(f for f in os.listdir(label_dir) if f.endswith(".txt"))
image_files = set(f for f in os.listdir(image_dir) if f.endswith(".jpg"))

# Check for orphaned files (label without image, image without label)
for f in label_files.union(image_files):
    label_path = os.path.join(label_dir, f.replace(".jpg", ".txt"))
    image_path = os.path.join(image_dir, f.replace(".txt", ".jpg"))

    label_exists = os.path.exists(label_path)
    image_exists = os.path.exists(image_path)

    if not label_exists or not image_exists:
        if label_exists:
            os.remove(label_path)
            print(f"🗑️ Removed orphan label: {label_path}")
        if image_exists:
            os.remove(image_path)
            print(f"🗑️ Removed orphan image: {image_path}")

# Now validate all remaining label files
for file in os.listdir(label_dir):
    if not file.endswith(".txt"):
        continue

    label_path = os.path.join(label_dir, file)
    image_path = os.path.join(image_dir, file.replace(".txt", ".jpg"))

    with open(label_path, "r") as f:
        lines = f.readlines()

    valid_lines = []
    for line in lines:
        try:
            parts = list(map(float, line.strip().split()))
            if (
                len(parts) == 5
                and 0 <= parts[1] <= 1 and 0 <= parts[2] <= 1
                and 0 < parts[3] <= 1 and 0 < parts[4] <= 1
            ):
                valid_lines.append(line)
        except:
            continue

    if valid_lines:
        with open(label_path, "w") as f:
            f.writelines(valid_lines)
    else:
        os.remove(label_path)
        if os.path.exists(image_path):
            os.remove(image_path)
            print(f"🧹 Removed image+label with no valid annotations: {file.replace('.txt', '')}")


In [ ]:
for file in os.listdir(label_dir):
    filepath = os.path.join(label_dir, file)
    with open(filepath, "r") as f:
        lines = f.readlines()
    valid_lines = []
    for line in lines:
        try:
            parts = list(map(float, line.strip().split()))
            if 0 <= parts[1] <= 1 and 0 <= parts[2] <= 1 and 0 < parts[3] <= 1 and 0 < parts[4] <= 1:
                valid_lines.append(line)
        except:
            continue
    if valid_lines:
        with open(filepath, "w") as f:
            f.writelines(valid_lines)
    else:
        os.remove(filepath)  # Delete file with no valid labels


Run this tomorrow morning, using the older yaml files to see if it doesn't mess up so badly.

In [13]:
from ultralytics import YOLO
import yaml

# Load your pre-trained YOLOv8 model
model = YOLO("yolov8m.pt")

# ✅ Load hyperparameters manually from hyp.yaml
with open(r"C:\Users\gnvca\OneDrive\Desktop\JP\model03split\satellite\hyp.yaml", "r") as f:
    hyp_params = yaml.safe_load(f)  # Converts YAML to dictionary

# ✅ Pass hyperparameters as individual keyword arguments
model.train(
    data=r"C:\Users\gnvca\OneDrive\Desktop\JP\model03split\satellite\yolo.yaml",  # Dataset config
    epochs=75,      # Total training epochs
    imgsz=512,       # Image size (try 768, 1024 for better accuracy)
    batch=16,        # Batch size (increase if GPU allows)
    project = 'ModelBuild',
    name='model03_yolo_drone',                 # Name of the specific experiment folder (creates a new folder)
    exist_ok=True,    
    amp=True,        # Automatic Mixed Precision (for speedup)
    conf=0.2,        # Confidence threshold for predictions
    iou=0.6,         # Intersection Over Union threshold
    optimizer="AdamW", # Optimizer choice
    device=0,        # Use GPU 0
    workers=4,       # Number of data workers (depends on CPU)
    cache=True,      # Cache images to RAM for faster training
    val=True,        # Run validation during training
    verbose=True,    # Print logs during training
    save=True,       # Save best and final models
)


Ultralytics 8.3.94  Python-3.10.16 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=C:\Users\gnvca\OneDrive\Desktop\JP\model03split\satellite\yolo.yaml, epochs=75, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=True, device=0, workers=4, project=ModelBuild, name=model03_yolo_drone, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.2, iou=0.6, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=

train: Scanning C:\Users\gnvca\OneDrive\Desktop\JP\model03split\satellite\dataset\labels\train.cache... 441 images, 0 backgrounds, 0 corrupt: 100%|██████████| 441/441 [00:00<?, ?it/s]

WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (0.3GB RAM): 100%|██████████| 441/441 [00:00<00:00, 2882.33it/s]
val: Scanning C:\Users\gnvca\OneDrive\Desktop\JP\model03split\satellite\dataset\labels\val.cache... 120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<?, ?it/s]

WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

                   all        120        218      0.583      0.506       0.57      0.269



75 epochs completed in 0.178 hours.
Optimizer stripped from ModelBuild\model03_yolo_drone\weights\last.pt, 52.0MB
Optimizer stripped from ModelBuild\model03_yolo_drone\weights\best.pt, 52.0MB

Validating ModelBuild\model03_yolo_drone\weights\best.pt...
Ultralytics 8.3.94  Python-3.10.16 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.21it/s]


                   all        120        218      0.596      0.547      0.592      0.277
                boiler         75         96       0.64      0.729      0.753      0.357
                 panel         54        122      0.552      0.364      0.432      0.196
Speed: 0.3ms preprocess, 4.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to ModelBuild\model03_yolo_drone


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022DC57A1210>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [9]:
from ultralytics import YOLO

# Load your trained model
model = YOLO(r"C:\Users\gnvca\OneDrive\Desktop\JP\ModelBuild\model03_yolo_drone\weights\best.pt")

# Export to ONNX
model.export(format="onnx")

Ultralytics 8.3.94  Python-3.10.16 torch-2.6.0+cu126 CPU (AMD Ryzen 5 7600 6-Core Processor)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from 'C:\Users\gnvca\OneDrive\Desktop\JP\ModelBuild\model03_yolo_drone\weights\best.pt' with input shape (1, 3, 512, 512) BCHW and output shape(s) (1, 6, 5376) (49.6 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success  1.5s, saved as 'C:\Users\gnvca\OneDrive\Desktop\JP\ModelBuild\model03_yolo_drone\weights\best.onnx' (98.8 MB)

Export complete (2.0s)
Results saved to C:\Users\gnvca\OneDrive\Desktop\JP\ModelBuild\model03_yolo_drone\weights
Predict:         yolo predict task=detect model=C:\Users\gnvca\OneDrive\Desktop\JP\ModelBuild\model03_yolo_drone\weights\best.onnx imgsz=512  
Validate:        yolo val task=detect model=C:\Users\gnvca\OneDrive\Desktop\JP\ModelBuild\model03_yolo_drone\weights\best.onnx imgsz=512 data=C:\Users\gnvca

'C:\\Users\\gnvca\\OneDrive\\Desktop\\JP\\ModelBuild\\model03_yolo_drone\\weights\\best.onnx'

In [10]:
from ultralytics import YOLO
import pandas as pd
import os

# Run inference on test images
results = list(model.predict(source=r"C:\Users\gnvca\OneDrive\Desktop\JP\model03\dataset\images\test", save=False))

# Debug: Check if results exist
print("Total images processed:", len(results))
if len(results) == 0:
    print("⚠️ No images processed! Check your dataset path.")
    




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1107 C:\Users\gnvca\OneDrive\Desktop\JP\model03\dataset\images\test\ID00qprY.jpg: 384x512 (no detections), 29.0ms
image 2/1107 C:\Users\gnvca\OneDrive\Desktop\JP\model03\dataset\images\test\ID01AciUc.jpg: 384x512 2 panels, 6.1ms
image 3/1107 C:\Users\gnvca\OneDrive\Desktop\JP\model03\dataset\images\test\ID0328D.jpg: 384x512 1 panel, 6.4ms
image 4/1107 C:\Users\gnvca\OneDrive\Desktop\JP\model03\dataset\images\test\ID05WxObCFTs9.jpg: 384x512 (no de

In [12]:
# Store results
rows = []

for result in results:
    image_id = os.path.splitext(os.path.basename(result.path))[0]  # Get filename without extension
    class_ids = result.boxes.cls.int().tolist()
    
    boiler_count = class_ids.count(0)  # assuming 'boiler' = class 0
    panel_count = class_ids.count(1)   # assuming 'panel' = class 1
    
    # Append each class as a separate row
    rows.append({"ID": f"{image_id}_boil", "Target": boiler_count})
    rows.append({"ID": f"{image_id}_pan", "Target": panel_count})

# Debug: Check if rows are being created
print("First 5 rows:", rows[:5])

# Create DataFrame and save CSV
df = pd.DataFrame(rows)

# If dataframe is empty, print a warning
if df.empty:
    print("⚠️ WARNING: DataFrame is empty! Check your detection logic.")

df.to_csv("test_detection_counts.csv", index=False)
print("✅ CSV saved as 'test_detection_counts.csv'")


First 5 rows: [{'ID': 'ID00qprY_boil', 'Target': 0}, {'ID': 'ID00qprY_pan', 'Target': 0}, {'ID': 'ID01AciUc_boil', 'Target': 0}, {'ID': 'ID01AciUc_pan', 'Target': 2}, {'ID': 'ID0328D_boil', 'Target': 0}]
✅ CSV saved as 'test_detection_counts.csv'


In [1]:
from ultralytics import YOLO
import pandas as pd
import os
import glob

# Load trained model
model = YOLO("runs/detect/train_safe4/weights/best.pt")

# Get all test images (ensure they're found)
image_files = glob.glob(r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\*")  # Any image format
if not image_files:
    raise ValueError("⚠️ No test images found! Check dataset path or file extensions.")

print(f"✅ Found {len(image_files)} test images.")

# Run inference on the test folder
results = model.predict(source=r"C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\\", save=False)

# Process results
rows = []
processed_images = set()

for result in results:
    image_id = os.path.splitext(os.path.basename(result.path))[0]
    processed_images.add(image_id)

    class_ids = result.boxes.cls.int().tolist()
    boiler_count = class_ids.count(0)
    panel_count = class_ids.count(1)

    rows.append({"ID": f"{image_id}_boil", "Target": boiler_count})
    rows.append({"ID": f"{image_id}_pan", "Target": panel_count})

# Ensure missing images have Target=0
for img_path in image_files:
    img_id = os.path.splitext(os.path.basename(img_path))[0]
    if img_id not in processed_images:
        rows.append({"ID": f"{img_id}_boil", "Target": 0})
        rows.append({"ID": f"{img_id}_pan", "Target": 0})

# Save CSV
df = pd.DataFrame(rows)
df.to_csv("test_detection_counts.csv", index=False)
print("✅ CSV saved: 'test_detection_counts.csv'")

✅ Found 1107 test images.


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1107 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID00qprY.jpg: 320x416 2 panels, 30.4ms
image 2/1107 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID01AciUc.jpg: 320x416 2 panels, 5.4ms
image 3/1107 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID0328D.jpg: 320x416 1 panel, 5.8ms
image 4/1107 C:\Users\gnvca\OneDrive\Desktop\JP\model03S\dataset\images\test\ID05WxObCF